In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE207847"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE207847"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE207847.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE207847.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE207847.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile of luminal breast cancer samples from FFPE sample of primary luminal breast cancer from patient who had a loco-regional recurrence [cohort A]"
!Series_summary	"We aimed at finding a gene signature able to discriminate patients who had a loco-regional recurrence either at early time after surgery or late (< 2 yrs, early;  >2 and <5 yrs, intermediate; < 5 yrs, late)"
!Series_overall_design	"We performed gene expression profile using Clariom D platform from FFPE sample of primary luminal breast cancer from patient who had a loco-regional recurrence at different time after surgery (cohort A)"
Sample Characteristics Dictionary:
{0: ['disease state: luminal breast cancer'], 1: ['gender: female'], 2: ['tissue: primary luminal breast cancer'], 3: ['loco-regional recurrence: LATE', 'loco-regional recurrence: EARLY', 'loco-regional recurrence: INTERMEDIATE']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this is a gene expression profile study using the Clariom D platform
# So this dataset is likely to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: The trait here refers to breast cancer, but specifically we are looking at the timing of 
# loco-regional recurrence which is in key 3
trait_row = 3  # loco-regional recurrence (EARLY, INTERMEDIATE, LATE)

# Age: There is no information about age in the sample characteristics
age_row = None

# Gender: Gender information is available in key 1
gender_row = 1  # gender: female

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert loco-regional recurrence data to binary format."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "EARLY" in value.upper():
        return 1  # Early recurrence (< 2 years)
    elif "LATE" in value.upper():
        return 0  # Late recurrence (> 5 years)
    elif "INTERMEDIATE" in value.upper():
        return 2  # Intermediate recurrence (2-5 years)
    else:
        return None

def convert_gender(value):
    """Convert gender data to binary format."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# Age function is not needed as age data is not available, but we'll define it for completeness
def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Process clinical data if trait_row is not None
if trait_row is not None:
    try:
        # Assuming clinical_data is a variable that contains the sample characteristics
        # from a previous step rather than a file to be loaded
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.txt")
        
        # Check if the variable clinical_data exists in the global namespace
        if 'clinical_data' in globals():
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age if age_row is not None else None,
                gender_row=gender_row,
                convert_gender=convert_gender if gender_row is not None else None
            )
            
            # Preview the data
            preview = preview_df(selected_clinical_df)
            print("Preview of clinical data:", preview)
            
            # Save to CSV
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Clinical data variable not found. Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")


Preview of clinical data: {'GSM6321013': [0.0, 0.0], 'GSM6321014': [1.0, 0.0], 'GSM6321015': [2.0, 0.0], 'GSM6321016': [2.0, 0.0], 'GSM6321017': [0.0, 0.0], 'GSM6321018': [0.0, 0.0], 'GSM6321019': [1.0, 0.0], 'GSM6321020': [0.0, 0.0], 'GSM6321021': [1.0, 0.0], 'GSM6321022': [1.0, 0.0], 'GSM6321023': [1.0, 0.0], 'GSM6321024': [0.0, 0.0], 'GSM6321025': [2.0, 0.0], 'GSM6321026': [1.0, 0.0], 'GSM6321027': [0.0, 0.0], 'GSM6321028': [2.0, 0.0], 'GSM6321029': [1.0, 0.0], 'GSM6321030': [0.0, 0.0], 'GSM6321031': [2.0, 0.0], 'GSM6321032': [0.0, 0.0], 'GSM6321033': [2.0, 0.0], 'GSM6321034': [2.0, 0.0], 'GSM6321035': [0.0, 0.0], 'GSM6321036': [1.0, 0.0], 'GSM6321037': [2.0, 0.0], 'GSM6321038': [2.0, 0.0], 'GSM6321039': [0.0, 0.0], 'GSM6321040': [2.0, 0.0], 'GSM6321041': [0.0, 0.0], 'GSM6321042': [0.0, 0.0], 'GSM6321043': [2.0, 0.0], 'GSM6321044': [0.0, 0.0], 'GSM6321045': [0.0, 0.0], 'GSM6321046': [2.0, 0.0], 'GSM6321047': [2.0, 0.0], 'GSM6321048': [1.0, 0.0], 'GSM6321049': [2.0, 0.0], 'GSM6321050

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE207847/GSE207847_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE207847/GSE207847_series_matrix.txt.gz
Found the matrix table marker at line 58


Gene data shape: (135750, 60)
First 20 gene/probe identifiers:
['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1', 'TC0100006437.hg.1', 'TC0100006438.hg.1', 'TC0100006439.hg.1', 'TC0100006440.hg.1', 'TC0100006441.hg.1', 'TC0100006442.hg.1', 'TC0100006443.hg.1', 'TC0100006444.hg.1', 'TC0100006445.hg.1', 'TC0100006446.hg.1', 'TC0100006447.hg.1', 'TC0100006448.hg.1', 'TC0100006449.hg.1', 'TC0100006450.hg.1', 'TC0100006451.hg.1']


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
gene_ids = ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1', 
           'TC0100006437.hg.1', 'TC0100006438.hg.1', 'TC0100006439.hg.1', 'TC0100006440.hg.1', 'TC0100006441.hg.1', 
           'TC0100006442.hg.1', 'TC0100006443.hg.1', 'TC0100006444.hg.1', 'TC0100006445.hg.1', 'TC0100006446.hg.1', 
           'TC0100006447.hg.1', 'TC0100006448.hg.1', 'TC0100006449.hg.1', 'TC0100006450.hg.1', 'TC0100006451.hg.1']

# These identifiers appear to be from the Affymetrix/Thermo Fisher Clariom D array (TC prefix)
# They are probeset IDs, not standard gene symbols
# These need to be mapped to standard human gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, the gene_assignment column contains gene symbols
print("\nExamining ID and gene_assignment columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'gene_assignment' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}, gene_assignment={gene_annotation['gene_assignment'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\ngene_assignment column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'category', 'locus type', 'SPOT_ID']
{'ID': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1'], 'probeset_id': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+'], 'start': ['11869', '28046', '29554'], 'stop': ['14412', '29178', '31109'], 'total_probes': [10.0, 6.0, 10.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000002844 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000362751 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain the gene identifiers and gene symbols
# From the previous step, we can see:
# - 'ID' column contains the probe identifiers (TC0100006432.hg.1, etc.)
# - 'gene_assignment' column contains gene symbols (e.g., DDX11L1, MIR1302-10)

# 2. Get a gene mapping dataframe by extracting the ID and gene_assignment columns
# The ID in gene_annotation corresponds to gene identifiers in the gene expression data
prob_col = 'ID'
gene_col = 'gene_assignment'

mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping dataframe:")
print(preview_df(mapping_df, n=5))

# 3. Apply gene mapping to convert probe-level data to gene-level data
# First check if we have the gene expression data from the previous step
if 'gene_data' not in locals():
    print("Extracting gene expression data...")
    gene_data_probes = get_genetic_data(matrix_file)
else:
    gene_data_probes = gene_data

# Apply the mapping to convert probe-level measurements to gene-level data
gene_data = apply_gene_mapping(gene_data_probes, mapping_df)
print(f"\nGene expression data shape after mapping: {gene_data.shape}")
print("First 5 genes and their expression values:")
print(preview_df(gene_data, n=5))

# Save gene data to CSV file for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (138745, 2)
First 5 rows of mapping dataframe:
{'ID': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000002844 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000362751 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'spopoybu.aAug10-unspliced // spopoybu // Transcript Identified by AceView // --- // ---', 'NR_036267 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000607096 // MIR1302-2 // microRNA 1302-2 // --- // 100302278', 'OTTHUMT00000471235 // OR4G4P // olfactory receptor, family 4, subfamily G, member 4 pseudogene // 1p36.33 // 79504', 'ENST00000492842 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // 15q26 // 26680 /// OTTHUMT000000


Gene expression data shape after mapping: (73312, 60)
First 5 genes and their expression values:
{'GSM6321013': [23.78333333333333, 13.778, 0.67375, 1.7553571428571428, 1.2], 'GSM6321014': [25.804166666666667, 13.702, 0.5725, 1.86, 1.3533333333333333], 'GSM6321015': [24.316666666666666, 12.75, 0.47125, 1.7839285714285715, 1.2366666666666666], 'GSM6321016': [24.875833333333333, 13.846, 0.70125, 1.8382142857142858, 1.2566666666666666], 'GSM6321017': [24.53, 13.372, 0.73375, 1.8485714285714285, 1.2566666666666666], 'GSM6321018': [23.740833333333335, 13.397, 0.5475, 1.7925, 1.2433333333333334], 'GSM6321019': [25.88, 13.86, 0.68625, 2.0039285714285713, 1.1966666666666665], 'GSM6321020': [24.820833333333333, 13.882, 0.71, 1.936785714285714, 1.0666666666666667], 'GSM6321021': [24.249166666666667, 13.501, 0.6475, 1.7717857142857143, 1.3399999999999999], 'GSM6321022': [25.4325, 14.193999999999999, 0.7875, 2.1514285714285712, 1.32], 'GSM6321023': [24.123333333333335, 13.218, 0.6575, 1.766428571


Gene expression data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE207847.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (73312, 60)
Gene data shape after normalization: (34003, 60)


Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE207847.csv


Extracted clinical data shape: (2, 60)
Preview of clinical data (first 5 samples):
               GSM6321013  GSM6321014  GSM6321015  GSM6321016  GSM6321017
Breast_Cancer         0.0         1.0         2.0         2.0         0.0
Gender                0.0         0.0         0.0         0.0         0.0
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE207847.csv
Gene data columns (first 5): ['GSM6321013', 'GSM6321014', 'GSM6321015', 'GSM6321016', 'GSM6321017']
Clinical data columns (first 5): ['GSM6321013', 'GSM6321014', 'GSM6321015', 'GSM6321016', 'GSM6321017']
Found 60 common samples between gene and clinical data
Initial linked data shape: (60, 34005)
Preview of linked data (first 5 rows, first 5 columns):
            Breast_Cancer  Gender   A1BG  A1BG-AS1    A1CF
GSM6321013            0.0     0.0  4.351     3.525  0.8300
GSM6321014            1.0     0.0  4.419     3.605  0.8800
GSM6321015            2.0     0.0  5.139     4.165  1.1150
GSM6321016      

Linked data shape after handling missing values: (60, 34005)
Quartiles for 'Breast_Cancer':
  25%: 0.0
  50% (Median): 1.0
  75%: 2.0
Min: 0.0
Max: 2.0
The distribution of the feature 'Breast_Cancer' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 60 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Breast_Cancer/GSE207847.csv
